In [4]:
import requests
from bs4 import BeautifulSoup
import re
import json
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [27]:
home = {'ucb': 'https://callink.berkeley.edu/Organizations', 
        'nau': 'https://nau.campuslabs.com/engage/organizations',
        'nyu': 'https://engage.nyu.edu/organizations',
        }

In [50]:
subpage = {'ucb': 'https://callink.berkeley.edu/organization/',
           'nau': 'https://nau.campuslabs.com/engage/organization/',
           'nyu': 'https://engage.nyu.edu/organization/',
        }

In [29]:
uni = 'nyu'

In [31]:
# initialize an instance of the chrome driver (browser)
driver = webdriver.Chrome()
# visit your target site
driver.get(home[uni])

In [32]:
# Load all pages
while driver.find_elements(By.TAG_NAME, 'button'):
    load_button = WebDriverWait(driver,30).until(EC.presence_of_element_located((By.TAG_NAME, 'button')))
    load_button.location_once_scrolled_into_view
    load_button.click()

In [52]:
# Grab all links
a_tags = driver.find_elements(By.TAG_NAME, 'a')
links = []
for a in a_tags:
    link = a.get_attribute('href')
    if link.startswith(subpage[uni]):
        links.append(link)

In [53]:
links

['https://engage.nyu.edu/organization/180-fellowship-all-university',
 'https://engage.nyu.edu/organization/1831fund',
 'https://engage.nyu.edu/organization/othmer-clark-halls',
 'https://engage.nyu.edu/organization/lafayette-hall',
 'https://engage.nyu.edu/organization/a-class-act',
 'https://engage.nyu.edu/organization/apabridge',
 'https://engage.nyu.edu/organization/academic-achievement-program',
 'https://engage.nyu.edu/organization/academy-of-general-dentistry-student-chapter-at-nyu-dentistry',
 'https://engage.nyu.edu/organization/ace-the-health-practitioner-s-society',
 'https://engage.nyu.edu/organization/aces-aros',
 'https://engage.nyu.edu/organization/active-minds-all-university',
 'https://engage.nyu.edu/organization/actuarial-society',
 'https://engage.nyu.edu/organization/advanced-nursing-student-organization',
 'https://engage.nyu.edu/organization/aesthetics-society',
 'https://engage.nyu.edu/organization/nyuagsa',
 'https://engage.nyu.edu/organization/african-heritage-

In [15]:
# 'description' is currently html code
fields = ['name', 'shortName', 'email', 'description', 'summary', 'status', 'socialMedia', 'primaryContact', 'profilePicture', 'organizationType']

In [16]:
def scrape(URL):
    data = {}
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    extract = soup.findAll('script')[8]
    text = extract.string[25:-1]

    dictionary = json.loads(text)
    orgData = dictionary['preFetchedData']['organization']
    for field in fields:
        data[field] = orgData[field]
    return data
    

In [56]:
example = scrape('https://engage.nyu.edu/organization/alpha-omega-epsilon')
for key in example:
    print(str(key) + ': ' + str(example[key]) + '\n')

name: Alpha Omega Epsilon

shortName: AOE

email: nyualphaomegaepsilon@gmail.com

description: <p>Alpha Omega Epsilon is a professional and social Sorority founded in 1983 that promotes friendship, leadership, and professionalism to all members of the organization, community, and our professions. The Beta Sigma Chapter here at NYU was founded on February 23rd, 2016 with a total of 23 founding members.</p>

summary: Alpha Omega Epsilon is a professional and social Sorority founded in 1983 that promotes friendship, leadership, and professionalism to all members of the organization, community, and our professions. The Beta Sigma Chapter here at NYU was founded on Fe...

status: Active

socialMedia: {'externalWebsite': 'https://aoenyu.wixsite.com/aoebetasigma', 'flickrUrl': '', 'googleCalendarUrl': '', 'googlePlusUrl': '', 'instagramUrl': 'https://instagram.com/aoenyu?igshid=NDA1YzNhOGU= ', 'linkedInUrl': '', 'pinterestUrl': '', 'tumblrUrl': '', 'vimeoUrl': '', 'youtubeUrl': '', 'facebookU

In [12]:
json_data = []
for link in links:
    json_data.append(scrape(link))

In [13]:
file_path = "nectar.json"

# Write the list to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(json_data, json_file)